In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from neo4j import GraphDatabase

In [9]:
df = pd.read_csv('/Users/stanleychen/Desktop/CompSci/Database/Project 1/hetionet/nodes.tsv', sep='\t')

In [13]:
display(df)

,id,name,kind
0,Anatomy::UBERON:0000002,uterine cervix,Anatomy
1,Anatomy::UBERON:0000004,nose,Anatomy
2,Anatomy::UBERON:0000006,islet of Langerhans,Anatomy
3,Anatomy::UBERON:0000007,pituitary gland,Anatomy
4,Anatomy::UBERON:0000010,peripheral nervous system,Anatomy
...,...,...,...
23031,Gene::9991,PTBP3,Gene
23032,Gene::9992,KCNE2,Gene
23033,Gene::9993,DGCR2,Gene
23034,Gene::9994,CASP8AP2,Gene


In [14]:
df2 = pd.read_csv('/Users/stanleychen/Desktop/CompSci/Database/Project 1/hetionet/edges.tsv', sep='\t')
display(df2)

,source,metaedge,target
0,Gene::801,GiG,Gene::7428
1,Gene::5987,GiG,Gene::9412
2,Gene::5747,GiG,Gene::79738
3,Gene::3725,GiG,Gene::10514
4,Gene::10014,GiG,Gene::55844
...,...,...,...
1292198,Anatomy::UBERON:0000057,AeG,Gene::65009
1292199,Anatomy::UBERON:0000474,AeG,Gene::80279
1292200,Anatomy::UBERON:0002048,AeG,Gene::1211
1292201,Anatomy::UBERON:0002048,AeG,Gene::8843
